# Postgres

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
import sys

### Connexion

In [4]:
def get_db_connection():
    """Créer la connexion à PostgreSQL via les variables d'environnement"""
    user = os.getenv('POSTGRES_USER')
    password = os.getenv('POSTGRES_PASSWORD') 
    host = os.getenv('POSTGRES_HOST', 'postgres')  # Seul défaut : nom service Docker
    port = os.getenv('POSTGRES_PORT')
    database = os.getenv('POSTGRES_DB')
    
    # Vérifier que toutes les variables sont définies
    if not all([user, password, port, database]):
        print("ERREUR : Variables d'environnement manquantes (POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_PORT, POSTGRES_DB)")
        sys.exit(1)
    
    connection_url = f'postgresql://{user}:{password}@{host}:{port}/{database}'
    
    try:
        engine = create_engine(connection_url, echo=False)
        return engine
    except Exception as e:
        print(f"Erreur de connexion à la base : {e}")
        sys.exit(1)


In [5]:
engine = get_db_connection()
print("✓ Connexion PostgreSQL établie")

✓ Connexion PostgreSQL établie


### Python -> Postrgres

In [6]:
def insert_new_records(engine):
    """Insérer deux nouveaux enregistrements dans la table iris"""
    
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COALESCE(MAX(id), 0) as max_id FROM iris"))
        max_id = result.scalar()
    
    new_data = pd.DataFrame({
        'id': [max_id + 1, max_id + 2],
        'sepal_length_cm': [5.9, 6.1],
        'sepal_width_cm': [2.6, 3.0], 
        'petal_length_cm': [4.2, 4.7],
        'petal_width_cm': [1.1, 1.7],
        'species': ['setosa', 'versicolor']
    })
    
    new_data.to_sql(
        name='iris',
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )
    
    print(f"✓ Insertion de {len(new_data)} nouveaux enregistrements réussie")

In [7]:
insert_new_records(engine)

✓ Insertion de 2 nouveaux enregistrements réussie


### Postrgres -> Python

In [14]:
import pandas as pd

def fetch_last_records(engine, limit=3):
    """Récupérer les derniers enregistrements de la table iris"""
    
    query = """
    SELECT id, sepal_length_cm, sepal_width_cm, 
           petal_length_cm, petal_width_cm, species
    FROM iris 
    ORDER BY id DESC 
    LIMIT %s
    """
    # Passage de 'limit' sous forme de tuple
    df_result = pd.read_sql_query(query, engine, params=(limit,))
    
    print(f"✓ Les {len(df_result)} derniers enregistrements :")
    return df_result


In [15]:
fetch_last_records(engine, limit=3)

✓ Les 3 derniers enregistrements :


,id,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,species
0,154,6.1,3.0,4.7,1.7,versicolor
1,153,5.9,2.6,4.2,1.1,setosa
2,152,6.2,3.1,4.8,1.8,versicolor


In [16]:
engine.dispose()
print("✓ Connexion fermée")

✓ Connexion fermée


# Mongo

In [17]:
import pymongo
from pymongo import MongoClient
import os
from datetime import datetime

In [18]:
# Configuration de connexion (adapter selon vos variables d'environnement)
MONGO_HOST = "mongodb_db"  # nom du service Docker
MONGO_PORT = os.getenv("MONGO_PORT")
MONGO_USERNAME = os.getenv("MONGO_ROOT_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_ROOT_PASSWORD")  
MONGO_DATABASE = os.getenv("MONGO_DB")

In [25]:
# Établir la connexion
def get_mongo_client():
    """Créer une connexion authentifiée vers MongoDB"""
    connection_string = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/{MONGO_DATABASE}?authSource=admin"
    return MongoClient(connection_string)


In [26]:
# Initialiser le client et la collection
client = get_mongo_client()
db = client[MONGO_DATABASE]
collection = db.iris

print(f"[INFO] Connexion établie vers la base {MONGO_DATABASE}")

[INFO] Connexion établie vers la base iris_db


### Python -> MongoDB

In [27]:
def insert_new_iris_samples():
    """Insérer deux nouveaux échantillons dans la collection iris"""
    
    # Récupérer l'ID maximum actuel pour générer les nouveaux IDs
    last_record = collection.find().sort("Id", -1).limit(1)
    max_id = list(last_record)[0]["Id"] if collection.count_documents({}) > 0 else 0
    
    # Définir les nouveaux échantillons
    new_samples = [
        {
            "Id": max_id + 1,
            "SepalLengthCm": 6.2,
            "SepalWidthCm": 3.8,
            "PetalLengthCm": 4.5,
            "PetalWidthCm": 1.3,
            "Species": "Iris-versicolor"
        },
        {
            "Id": max_id + 2,
            "SepalLengthCm": 7.1,
            "SepalWidthCm": 2.9,
            "PetalLengthCm": 5.8,
            "PetalWidthCm": 2.1,
            "Species": "Iris-virginica"
        }
    ]
    
    # Exécuter l'insertion
    try:
        result = collection.insert_many(new_samples)
        print(f"[SUCCESS] {len(result.inserted_ids)} échantillons insérés")
        print(f"[INFO] IDs générés: {result.inserted_ids}")
        return result.inserted_ids
    except Exception as e:
        print(f"[ERROR] Échec de l'insertion: {str(e)}")
        return None


In [28]:
# Exécuter l'insertion
inserted_ids = insert_new_iris_samples()

[SUCCESS] 2 échantillons insérés
[INFO] IDs générés: [ObjectId('685e45f9bdaf3b9965d5dd6b'), ObjectId('685e45f9bdaf3b9965d5dd6c')]


### MongoDB -> Python

In [29]:
def get_last_iris_samples(limit=3):
    """Récupérer les N derniers échantillons de la collection"""
    
    try:
        # Trier par ID décroissant et limiter le résultat
        cursor = collection.find().sort("Id", -1).limit(limit)
        samples = list(cursor)
        
        print(f"[INFO] {len(samples)} derniers échantillons récupérés:")
        print("-" * 80)
        
        for i, sample in enumerate(samples, 1):
            print(f"Échantillon {i}:")
            print(f"  ID: {sample['Id']}")
            print(f"  Sepal: {sample['SepalLengthCm']}cm x {sample['SepalWidthCm']}cm")
            print(f"  Petal: {sample['PetalLengthCm']}cm x {sample['PetalWidthCm']}cm")
            print(f"  Espèce: {sample['Species']}")
            print(f"  ObjectId: {sample['_id']}")
            print("-" * 40)
            
        return samples
        
    except Exception as e:
        print(f"[ERROR] Échec de la récupération: {str(e)}")
        return None



In [30]:
# Récupérer les 3 derniers échantillons
last_samples = get_last_iris_samples(3)

[INFO] 3 derniers échantillons récupérés:
--------------------------------------------------------------------------------
Échantillon 1:
  ID: 154
  Sepal: 7.1cm x 2.9cm
  Petal: 5.8cm x 2.1cm
  Espèce: Iris-virginica
  ObjectId: 685e45f9bdaf3b9965d5dd6c
----------------------------------------
Échantillon 2:
  ID: 153
  Sepal: 6.2cm x 3.8cm
  Petal: 4.5cm x 1.3cm
  Espèce: Iris-versicolor
  ObjectId: 685e45f9bdaf3b9965d5dd6b
----------------------------------------
Échantillon 3:
  ID: 152
  Sepal: 7.1cm x 2.9cm
  Petal: 5.8cm x 2.1cm
  Espèce: virginica
  ObjectId: 685d69040c0b099912335434
----------------------------------------


In [31]:
client.close()
print("[INFO] Connexion fermée")

[INFO] Connexion fermée


# Test Ollama

In [ ]:
!conda install requests -q -y

In [6]:
import requests

def test_ollama_connection():
    try:
        # Utiliser l'adresse IP du service Ollama ou le nom d'hôte correct
        response = requests.get("http://ollama:11434/api/tags")
        if response.status_code == 200:
            models = response.json()
            print("✅ Connexion Ollama réussie")
            print("Modèles disponibles:", [model['name'] for model in models.get('models', [])])
        else:
            print("❌ Erreur de connexion:", response.status_code)
    except Exception as e:
        print("❌ Erreur de connexion:", e)

test_ollama_connection()

✅ Connexion Ollama réussie
Modèles disponibles: ['phi4-mini:latest']


In [7]:
import requests
import json

# Test de connexion à Ollama
def test_ollama():
    url = "http://ollama:11434/api/generate"
    data = {
        "model": "phi4-mini:latest", # phi4-mini:latest  granite3-dense:2b
        "prompt": "Peux tu te présenter?",
        "stream": False
    }
    
    response = requests.post(url, json=data)
    if response.status_code == 200:
        return response.json()['response']
    else:
        return f"Erreur: {response.status_code}"

# Utilisation
result = test_ollama()
print(result)


Je m'appelle Phi, une intelligence artificielle développée par Microsoft pour vous aider à répondre aux questions et fournir des informations. Comment puis-je vous assister aujourd'hui ?


# MINIO

In [ ]:
from minio import Minio
from minio.error import S3Error
from PIL import Image
import io

# 1. Créer une image noire (100x100)
img = Image.new('RGB', (100, 100), color='black')
buffer = io.BytesIO()
img.save(buffer, format="JPEG")
buffer.seek(0)

# 2. Connexion au client MinIO
client = Minio(
    endpoint="localhost:9000",  # Adapter si conteneur
    access_key="minioadmin",
    secret_key="minioadmin123",
    secure=False
)

# 3. Créer le bucket s'il n'existe pas
bucket_name = "images"
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)

# 4. Upload de l'image noire
object_name = "generated_image.jpg"
client.put_object(
    bucket_name=bucket_name,
    object_name=object_name,
    data=buffer,
    length=buffer.getbuffer().nbytes,
    content_type="image/jpeg"
)

print(f"✅ Image '{object_name}' uploadée dans le bucket '{bucket_name}'")
